In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
import joblib
from datetime import datetime
import concurrent

BASE_DIR = "../"
PASQUINIS_PATH = BASE_DIR + "traces-netsoft-2017"
DATE = datetime.now().isoformat(timespec='seconds')
BASE_RESULTS_PATH = f'./resultados_pre_pesquisa/{DATE}'
MODELS_DIR = "models/"
MODELS_PATH_PREFIX = f'models/{DATE}_'

TRACES=[
    #"KV-BothApps-FlashcrowdLoad",
    #"KV-BothApps-PeriodicLoad",
    #"KV-SingleApp-FlashcrowdLoad",
    #"KV-SingleApp-PeriodicLoad",
    #"VoD-BothApps-FlashcrowdLoad",
    #"VoD-BothApps-PeriodicLoad",
    #"VoD-SingleApp-FlashcrowdLoad",
    "VoD-SingleApp-PeriodicLoad"]

NROWS = None
PERSIST = False
Y_METRIC = 'DispFrames'
VOD_SINGLEAPP_PERIODIC_LOAD_PATH = f'{PASQUINIS_PATH}/VoD-SingleApp-PeriodicLoad'

def nmae(y_pred, y_test):
    return abs(y_pred - y_test).mean() / y_test.mean()

if PERSIST:
    results_path = f'{BASE_RESULTS_PATH}'
    try:
        os.makedirs(results_path)
        os.makedirs(MODELS_DIR)
    except FileExistsError:
        print("Já criado diretório...")

In [3]:
y_dataset = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/Y.csv', header=0, nrows=NROWS, index_col=0, usecols=['TimeStamp', 'DispFrames'], low_memory=True).apply(pd.to_numeric, errors='coerce').fillna(0)

x_files = ['X_cluster.csv', 'X_flow.csv', 'X_port.csv']

x_trace = pd.DataFrame()

for x_file in x_files:
    read_dataset = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                              header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)
    if len(x_trace.columns) != 0:
        x_trace.merge(read_dataset, how="inner",
            on="TimeStamp", copy=False)
    else:
        x_trace = read_dataset



In [12]:
X_train, X_test, y_train, y_test = train_test_split(x_trace, y_dataset, test_size=0.7, random_state=42)


In [8]:
import time

In [21]:

regression_tree = DecisionTreeRegressor() 

tempo_reg_tree = time.time()
regression_tree.fit(X_train, y_train)
tempo_reg_tree = time.time() - tempo_reg_tree

if PERSIST:
    joblib.dump(regression_tree, f'{MODELS_PATH_PREFIX}_model_regression-tree.sav')

print(f'Training Time Reg Tree: {tempo_reg_tree}')
pred_reg_tree = regression_tree.predict(X_test)
print(f'NMAE: {nmae(pred_reg_tree, y_test["DispFrames"])}')

Training Time Reg Tree: 32.23722505569458
NMAE: 0.10112599697648961


In [25]:
regr_random_forest = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest = time.time()
regr_random_forest.fit(X_train, y_train)
tempo_random_forest = time.time() - tempo_random_forest

if PERSIST:
    joblib.dump(regr_random_forest, f'{MODELS_PATH_PREFIX}_model_random-forest.sav')

print(f'Training Time Random Forest: {tempo_random_forest}')
pred_random_forest = regr_random_forest.predict(X_test)
print(f'NMAE: {nmae(pred_random_forest, y_test["DispFrames"])}')

/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest: 320.60004210472107


In [26]:
print(f'NMAE: {nmae(pred_random_forest, y_test["DispFrames"])}')

NMAE: 0.0934177251849149


In [28]:
## para X_flow ou x_port

x_file = 'X_flow.csv' 

print(f'Only dataset {x_file}')
x_trace_minimal_flow = pd.DataFrame()

x_trace_minimal_flow = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                          header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)

X_train_flow, X_test_flow, y_train_flow, y_test_flow = train_test_split(x_trace_minimal_flow, y_dataset, test_size=0.7, random_state=42)

regression_tree_flow = DecisionTreeRegressor() 

tempo_reg_tree_flow = time.time()
regression_tree_flow.fit(X_train_flow, y_train_flow)
tempo_reg_tree_flow = time.time() - tempo_reg_tree_flow

if PERSIST:
    joblib.dump(regression_tree_flow, f'{MODELS_PATH_PREFIX}_model_regression-tree_for-{x_file}.sav')

print(f'Training Time only {x_file} Reg Tree: {tempo_reg_tree_flow}')
pred_reg_tree_flow = regression_tree_flow.predict(X_test_flow)
print(f'NMAE only {x_file}: {nmae(pred_reg_tree_flow, y_test["DispFrames"])}')

regr_random_forest_flow = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_flow = time.time()
regr_random_forest_flow.fit(X_train_flow, y_train_flow)
tempo_random_forest_flow = time.time() - tempo_random_forest_flow

if PERSIST:
    joblib.dump(regr_random_forest_flow, f'{MODELS_PATH_PREFIX}_model_random-forest_for-{x_file}.sav')

print(f'Training Time Random Forest only {x_file}: {tempo_random_forest_flow}')
pred_random_forest_flow = regr_random_forest_flow.predict(X_test_flow)
print(f'NMAE only {x_file}: {nmae(pred_random_forest_flow, y_test["DispFrames"])}')



Only dataset X_flow.csv
Training Time only X_flow.csv Reg Tree: 0.3066692352294922
NMAE only X_flow.csv: 0.11325286084708491


/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest only X_flow.csv: 3.2016818523406982
NMAE only X_flow.csv: 0.10733201366994324


In [29]:
#'X_port.csv'

x_file = 'X_port.csv' 

print(f'Only dataset {x_file}')
x_trace_minimal_port = pd.DataFrame()

x_trace_minimal_port = pd.read_csv(f'{VOD_SINGLEAPP_PERIODIC_LOAD_PATH}/{x_file}', 
                          header=0, index_col=0, low_memory=True, nrows=NROWS).apply(pd.to_numeric, errors='coerce').fillna(0)

X_train_port, X_test_port, y_train_port, y_test_port = train_test_split(x_trace_minimal_port, y_dataset, test_size=0.7, random_state=42)

regression_tree_port = DecisionTreeRegressor() 

tempo_reg_tree_port = time.time()
regression_tree_port.fit(X_train_port, y_train_port)
tempo_reg_tree_port = time.time() - tempo_reg_tree_port

if PERSIST:
    joblib.dump(regression_tree_port, f'{MODELS_PATH_PREFIX}_model_regression-tree_for-{x_file}.sav')

print(f'Training Time only {x_file} Reg Tree: {tempo_reg_tree_port}')
pred_reg_tree_port = regression_tree_port.predict(X_test_port)
print(f'NMAE only {x_file}: {nmae(pred_reg_tree_port, y_test["DispFrames"])}')

regr_random_forest_port = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)

tempo_random_forest_port = time.time()
regr_random_forest_port.fit(X_train_port, y_train_port)
tempo_random_forest_port = time.time() - tempo_random_forest_port

if PERSIST:
    joblib.dump(regr_random_forest_port, f'{MODELS_PATH_PREFIX}_model_random-forest_for-{x_file}.sav')

print(f'Training Time Random Forest only {x_file}: {tempo_random_forest_port}')
pred_random_forest_port = regr_random_forest_port.predict(X_test_port)
print(f'NMAE only {x_file}: {nmae(pred_random_forest_port, y_test["DispFrames"])}')



Only dataset X_port.csv
Training Time only X_port.csv Reg Tree: 2.628509044647217
NMAE only X_port.csv: 0.11026255886288207


/home/davi/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Training Time Random Forest only X_port.csv: 25.735234260559082
NMAE only X_port.csv: 0.10501668143668873


In [ ]:
#minimal dataset creation
import warnings 

warnings.filterwarnings('ignore')

x_trace_minimal = pd.DataFrame()
old_reg_tree_nmae = 1
old_random_forest_nmae = 1

for column in x_trace.columns:
    x_trace_minimal[column] = x_trace[[column]].copy()
    
    X_train_minimal , X_test_minimal, y_train_minimal, y_test_minimal = train_test_split(x_trace_minimal, y_dataset, test_size=0.7, random_state=42)
    
    regression_tree_minimal = DecisionTreeRegressor() 
    regression_tree_minimal.fit(X_train_minimal, y_train_minimal)
    pred_reg_tree_minimal = regression_tree_minimal.predict(X_test_minimal)
    
    new_nmae_reg_tree = nmae(pred_reg_tree_minimal, y_test["DispFrames"])

    random_forest_minimal = RandomForestRegressor(n_estimators=120, random_state=42, n_jobs=-1)
    random_forest_minimal.fit(X_train_minimal, y_train_minimal)
    pred_random_forest_minimal = random_forest_minimal.predict(X_test_minimal)
    
    new_nmae_random_forest = nmae(pred_random_forest_minimal, y_test["DispFrames"])
     
    if old_reg_tree_nmae >= new_nmae_reg_tree or old_random_forest_nmae >= new_nmae_random_forest:
        old_reg_tree_nmae = new_nmae_reg_tree if new_nmae_reg_tree < old_reg_tree_nmae else old_reg_tree_nmae
        old_random_forest_nmae = new_nmae_random_forest if new_nmae_random_forest < old_random_forest_nmae else old_random_forest_nmae
    else:
        x_trace_minimal.drop([column], axis=1, inplace=True)
    